In [ ]:
#Import ultralytics
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
# Import required libraries
import os
import shutil
import random

#!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
# Connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Assign model and test images
from ultralytics import YOLO
model=YOLO('/content/drive/MyDrive/UU_Thesis/weights/best.pt')
source='/content/drive/MyDrive/UU_Thesis/selected_images'
print(type(model))

<class 'ultralytics.yolo.engine.model.YOLO'>


In [ ]:
#Predict images
results = model(source)


image 1/93 /content/drive/MyDrive/UU_Thesis/selected_images/1000.JPG: 640x480 2 Goats, 126.5ms
image 2/93 /content/drive/MyDrive/UU_Thesis/selected_images/1003.jpg: 640x480 3 Goats, 28.2ms
image 3/93 /content/drive/MyDrive/UU_Thesis/selected_images/1010.JPG: 480x640 5 Goats, 108.2ms
image 4/93 /content/drive/MyDrive/UU_Thesis/selected_images/1019.jpg: 480x640 8 Goats, 28.2ms
image 5/93 /content/drive/MyDrive/UU_Thesis/selected_images/1047.jpg: 320x640 4 Goats, 113.9ms
image 6/93 /content/drive/MyDrive/UU_Thesis/selected_images/1049.jpg: 640x640 5 Goats, 37.9ms
image 7/93 /content/drive/MyDrive/UU_Thesis/selected_images/1068.jpg: 352x640 2 Goats, 73.3ms
image 8/93 /content/drive/MyDrive/UU_Thesis/selected_images/1084.jpg: 640x480 8 Goats, 28.8ms
image 9/93 /content/drive/MyDrive/UU_Thesis/selected_images/1093.jpg: 448x640 5 Goats, 69.2ms
image 10/93 /content/drive/MyDrive/UU_Thesis/selected_images/1131.jpg: 640x384 14 Goats, 67.0ms
image 11/93 /content/drive/MyDrive/UU_Thesis/selected_

# Storing results

In [ ]:
# Set the directory where the file will be saved
directory = "/content/drive/MyDrive/UU_Thesis/output/boxes"
if not os.path.exists(directory):
  os.makedirs(directory)

for n, result in enumerate(results):
    boxes = str(result.boxes.xywh)     #xyxy format /or xywh format
    #print("For result ", n, result,"data =",boxes)
    #print("Boxes Conf = ", result.boxes.conf)

    #Get image path
    path = result.path
    filename = path[path.rfind("/") + 1:]

    new_extension = '.txt'

    if '.' in filename:
        # Split the filename into the base name and extension
        parts = filename.rsplit('.', 1)

        # Replace the extension with the new extension
        new_filename = f"{parts[0]}{new_extension}"
    else:
        # If the filename doesn't have an extension, simply append the new extension to the end
        new_filename = f"{filename}{new_extension}"

    #print(new_filename)

    # Construct the full path to the new file
    new_file_path = os.path.join(directory, new_filename)

    # Open the new file for writing
    with open(new_file_path, "w") as f:
        # Write the contents of 'boxes' to the file
        f.write(boxes)

## Segment Anything

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

%cd {HOME}

# Install Segment Anything
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

# Install jupyter widget
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

# Retrieve weights from Meta
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

# Create data folder
%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data

HOME: /content
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-l3rgx8ex
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-l3rgx8ex
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=53747b2af9e09e850f080a835fc8339ae3ec772a39e64f99fece5e36283ab3d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6n955yte/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.3 MB/s 

Load model

In [ ]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

##Loading mask predictor

In [ ]:
mask_predictor = SamPredictor(sam)

##Loading image

In [ ]:
def yolo_convertor(yolo_tensor):
    boxes = []
    for box in yolo_tensor:
        print(box)  # Add this line to inspect the contents of 'box'
        x_center, y_center, width, height, *_ = box  # Use *_ to ignore extra values
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        x2 = x_center + width / 2
        y2 = y_center + height / 2
        boxes.append([x1, y1, x2, y2])

    boxes = np.array(boxes)
    return boxes

In [ ]:
import os
import cv2
import numpy as np
import re

# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

path_to_selected_images = "/content/drive/MyDrive/UU_Thesis/selected_images"
path_to_bounding_boxes = "/content/drive/MyDrive/UU_Thesis/output/boxes"

def load_images_and_tensors():
    image_files = os.listdir(path_to_selected_images)
    text_files = os.listdir(path_to_bounding_boxes)

    # Select only text files that have a corresponding image file name without checking the extension
    text_files = [text_file for text_file in text_files if os.path.splitext(text_file)[0] in image_files]

    data = []

    for image_file in image_files:
        # Load image
        image_path = os.path.join(path_to_selected_images, image_file)
        image = cv2.imread(image_path)

        # Load corresponding text file
        text_file = os.path.splitext(image_file)[0] + '.txt'
        text_path = os.path.join(path_to_bounding_boxes, text_file)
        print("text_path:", text_path)

        with open(text_path, 'r') as f:
            tensor_str = f.read()
            # Remove non-numeric characters and extra spaces
            tensor_str = re.sub(r'[^0-9.,\s]', '', tensor_str).strip()
            # Convert string to numpy array
            try:
              tensor = np.array([list(map(float, line.split(','))) for line in tensor_str.split('\n') if line.strip()])
            except:
              continue


        data.append({'image': image, 'tensor': tensor, 'image_path': image_path})

    return data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
yolo_test_data = load_images_and_tensors()
yolo_convertor(yolo_test_data[2]['tensor'])[0]

text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/29.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/1177.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/63.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/764.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/1049.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/752.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/968.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/335.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/69.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/863.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/18.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/1414.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/652.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/1003.txt
text_path: /content/drive/MyDrive/UU_Thesis/output/boxes/509.txt
text_path: /content/drive

array([     34.177,      292.16,      1204.5,      1290.9])

In [ ]:
len(yolo_test_data
  )

13

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def seperate(detections, image_bgr, image):

  # Get the mask
  mask = detections.mask
  # Squeeze any singleton dimensions (if there's only one mask)
  mask = np.squeeze(mask)

  # Ensure the mask is a boolean mask
  mask = mask.astype(bool)

  # Create a new image with 4 channels (RGB + alpha)
  cutout = np.zeros((image_bgr.shape[0], image_bgr.shape[1], 4), dtype=np.uint8)

  # Set RGB values
  cutout[..., :3] = image_bgr

  # Set alpha channel
  cutout[..., 3] = mask.astype(np.uint8) * 255  # Set alpha to 255 for non-transparent regions

  # Convert the image from BGR to RGBA
  cutout_rgba = cv2.cvtColor(cutout, cv2.COLOR_BGRA2RGBA)

  # Save the image with transparency to a file (e.g., PNG)
  output_file = 'output.png'
  img = Image.fromarray(cutout_rgba)

  # Save image to google drive

  image_path = "/content/drive/MyDrive/UU_Thesis/segmented_images/CutOut"
  filename = os.path.basename(image['image_path'])
  output_path = os.path.join(image_path, filename)
  # Remove existing file if it exists within the target directory
  if os.path.exists(output_path) and os.path.isfile(output_path) and 'Thesis' in os.path.dirname(output_path):
      os.remove(output_path)

  img.save(output_path, "PNG")
  # return

In [ ]:
import supervision as sv

In [ ]:
#1
import numpy as np
import cv2
import supervision as sv
from jupyter_bbox_widget import BBoxWidget
widget = BBoxWidget()

# default_box is going to be used if you will not draw any box on image above
default_box = {'x': 68, 'y': 247, 'width': 555, 'height': 678, 'label': ''}

for image in yolo_test_data:
    box = widget.bboxes[0] if widget.bboxes else default_box
    box = yolo_convertor(image['tensor'])[0]

    image_bgr = cv2.imread(image['image_path'])
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    mask_predictor.set_image(image_rgb)

    masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True
    )


    #box_annotator = sv.BoxAnnotator() #color=sv.Color.red()
    mask_annotator = sv.MaskAnnotator(color=sv.Color(0,0,0)) #color=sv.Color.red()

    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    #print(detections)
    #with open('readmeDetection123.txt', 'w') as f:
    #    f.writelines(detections)
    detections = detections[detections.area == np.max(detections.area)]

    #source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections2, skip_label=True)
    segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

    seperate(detections, image_bgr, image)

    # Save image to google drive
    cv2.imwrite(os.path.join("/content/drive/MyDrive/UU_Thesis/segmented_images", os.path.basename(image['image_path'])), segmented_image) #segmented_image


[     224.37      188.22      442.04      376.44           0]
[     165.93      160.21      184.07      208.88           0]
[     619.32      791.53      1170.3      998.75           0]
[      150.4      148.32      289.11      296.52           0]
[     339.99      242.16      284.69      307.47           0]
[     327.32      279.93      641.77         533           0]
[     497.08      618.56       529.5      588.38           0]
[      170.7      151.94      238.31       295.2           0]
[     324.22      315.68      516.72      604.92           0]
[     537.97      667.16      529.68      583.32           0]
[     531.94      425.76      146.39       155.8           0]
[     960.82      808.96      1802.1      1524.4           0]
[     827.84      591.55      1282.3      1149.6           0]
